In [1]:
import numpy as np
import gurobipy as gb
import math as m
from gurobipy import GRB

<b style='color:red;'><strong>Условие</strong></b>

Продукт 1 более ценный

В день:

до 100000 т по цене 3.25 руб/т - руды А

до 30000 т по цене 3.40 руб/т - руды B

100000 при затратах 0.35 руб/т - Общая мощность основного процесса переработки

Из 1 т А основной способ позваоляет получить 0.15 т продукта 1 и 0.85 продукта 2

Из 1 т B основной способ позваоляет получить 0.25 т продукта 1 и 0.75 продукта 2

Из 1 т продукта 2 агрегатом можно получить 0.5 т продукта 1 и 0.5 т продукта 2

Мощность конфертора 50000 т в день при затратах 0.25 руб/т

Затраты на фильтрацию продукта 1 после основного процесса обработки 0.10 руб/т

Продукт 2 может быть реализован неограничено по цене 3.8 руб/т

Продукт 1 продается по 5.5 руб/т можно продать до 45000 т/день до 4000 т/день по 5.2 руб/т и неограниченое по 5 руб/т

В день не меньше 40000 тон продукта 1

Закупка продукта 1 5.75 руб/т

Закупка продукта 2 4 руб/т

<b style='color:red;'><strong> Ввести переменные, построить математическую модель и </strong></b>
    
1)найти план выпуска продукции с максимальной прибылью;
    
2)найти минимальную цену на продукт I, при которой прибыль предприятия будет не меньше 50 тыс. руб./день.
    
3)как изменится оптимальный план, если мощность фильтра ограничена величиной 10 тыс. тонн?
    
4)как изменится оптимальный план производства, если фильтр выйдет из строя, и продукт I можно будет продавать только по цене 5 руб./т.? 


<b style='color:red;'><strong>Мат модель</strong></b>

<b style='color:green;'><strong>Договоримся о пременных:</strong></b>

$A_W$ - масса в день руды A

$B_W$ - масса в день руды B

$A_S$ - цена поставки руды А

$B_S$ - цена поставки руды B

$W$ - мощность основного процесса переработки

$C$ - затраты на переработку основным способом

$A_{p1} $ - вес 1 продукта полученного из руды А основным способом

$A_{p2} $ - вес 2 продукта полученного из руды А основным способом

$B_{p1} $ - вес 1 продукта полученного из руды B основным способом

$B_{p2} $ - вес 2 продукта полученного из руды B основным способом

$Con_W$ - мощность конвертора

$Con_S$ - затраты на обработку конвертором

$Con_{p1}$ - сколько продукта 1 можно получить из конвертора

$Con_{p2}$ - сколько продукта 2 можно получить из ковертора

$Filt_{p1}$ - затраты на фильтрацию продукта 1 после основного процесса

$p2_c$ - цена реализации продукта 2

Пусть $H = \begin{cases}1 - \text{если продукт 1 продают по самой высокой цене} \\0 -\text{иначе} \end{cases}$

Пусть $M = \begin{cases}1 - \text{если продукт 1 продают по средней цене} \\0 -\text{иначе} \end{cases}$

$p1_{Hc}$ - самая высокая цена, по которой можно реализовать продукт 1

$p1_{Mc}$ - средняя цена за которую можно реализовать продукт 1

$p1_{Sc}$ - самая низкая цена по, которой можно реализвать продукт 1

z - сколько конвертировали 2 изделия

x - сколько купили руды А

y - сколько купили руды B

g - сколько продукта 1 продали по самой высокой цене

k - сколько продукта 1 продали по средней цене

p - сколько изделия 1 докупили

f - сколько изделия 2 докупили

<b style='color:orange;'><strong>Ограничения:</strong></b>

z $\leqslant$ Con_W

x + y <= W

(x * A_p1 + y * B_p1 + z * Con_p1) + p $\geqslant$ p1_Hm

(x * A_p1 + y * B_p1 + z * Con_p1) + p - g - k $\geqslant$ 0

z $\leqslant$ x * A_p2 + y * B_p2

x $\leqslant$ A_w

y $\leqslant$ B_w

g $\leqslant$ H

k $\leqslant$ M

x $\geqslant$ 0

y $\geqslant$ 0

g $\geqslant$ 0

k $\geqslant$ 0

p $\geqslant$ 0

f $\geqslant$ 0

z $\geqslant$ 0



<b style='color:blue;'><strong>Целевая функция:</strong></b>

$((z * Con_p2 + (x * a_p2 + y * b_p2 + f) - z) * p2_c + g * p1_Hc + k * p1_Mc + ((x * a_p1 + y * b_p1 + z * Con_p1) + p - g - k) * p1_Sc) - 
               (x * A_S + y * B_S + (x + y) * С + z * Con_S + (x * a_p1 + y * b_p1 + z * Con_p1) * Filt_c + f * p2_p + p * p1_p) \to \max$

((Цена реализации продукта 2 + цена реализации продукта 1 H + цена реализации продукта 1 M + реализация продукта 1 S) - (закупка руда А + закупка руды В + затраты на переработку основным способом + затраты на конвертор + затраты на фильтрацию + закупка руды 2 + закупка руды 1))$\to \max$

In [2]:
A_S = 3.25
B_S = 3.40
A_w = 100000
B_w = 30000
W = 100000
С = 0.35
A_p1 = 0.15
A_p2 = 0.85
B_p1 = 0.25
B_p2 = 0.75
Con_W = 50000
Con_S = 0.25
Con_p1 = 0.5
Con_p2 = 0.5
Filt_c = 0.1
p2_c = 3.80
H = 45000
M = 4000
p1_Hc = 5.50
p1_Mc = 5.2
p1_Sc = 5.0
P1 = 40000
P2 = 4000
p1_p = 5.75
p2_p = 4.00

In [16]:
a = gb.Model("fabric")

x = a.addVar(vtype = GRB.INTEGER, name = "buyRa")
y = a.addVar(vtype = GRB.INTEGER, name = "buyRb")
g = a.addVar(vtype = GRB.INTEGER, name = "sellH")
k = a.addVar(vtype = GRB.INTEGER, name = "sellM")
p = a.addVar(vtype = GRB.INTEGER, name = "buy1")
f = a.addVar(vtype = GRB.INTEGER, name = "buy2")
z = a.addVar(vtype = GRB.INTEGER, name = "Convert2")

obj_= x * A_p2 + y * B_p2 + f
obj1 = x * A_p1 + y * B_p1 + z * Con_p1
obj2 = z * Con_p2 + obj_ - z

a.setObjective((obj2 * p2_c + g * p1_Hc + k * p1_Mc + (obj1 + p - g - k) * p1_Sc) - 
               (x * A_S + y * B_S + (x + y) * С + z * Con_S + (obj1 -z * Con_p1)  * Filt_c + f * p2_p + p * p1_p), GRB.MAXIMIZE)

a.addConstr(z <= Con_W)

a.addConstr(x + y <= W)

a.addConstr(obj1 + p >= P1)
a.addConstr(obj1 + p - g - k >= 0)
a.addConstr(obj2>= P1)
a.addConstr(z <= obj_)
a.addConstr(x <= A_w)
a.addConstr(y <= B_w)
a.addConstr(g <= H)
a.addConstr(k <= M)

a.addConstr(x >= 0)
a.addConstr(y >= 0)
a.addConstr(g >= 0)
a.addConstr(k >= 0)
a.addConstr(p >= 0)
a.addConstr(f >= 0)
a.addConstr(z >= 0)

a.update()
print(a.display())
a.optimize()

Maximize
0.365 buyRa + 0.32499999999999973 buyRb + 0.5 sellH + 0.20000000000000018 sellM +
-0.75 buy1 + -0.20000000000000018 buy2 + 0.3500000000000001 Convert2
Subject To
  R0: Convert2 <= 50000
  R1: buyRa + buyRb <= 100000
  R2: 0.15 buyRa + 0.25 buyRb + buy1 + 0.5 Convert2 >= 40000
  R3: 0.15 buyRa + 0.25 buyRb + -1.0 sellH + -1.0 sellM + buy1 + 0.5 Convert2 >= 0
  R4: 0.85 buyRa + 0.75 buyRb + buy2 + -0.5 Convert2 >= 40000
  R5: -0.85 buyRa + -0.75 buyRb + -1.0 buy2 + Convert2 <= 0
  R6: buyRa <= 100000
  R7: buyRb <= 30000
  R8: sellH <= 45000
  R9: sellM <= 4000
  R10: buyRa >= 0
  R11: buyRb >= 0
  R12: sellH >= 0
  R13: sellM >= 0
  R14: buy1 >= 0
  R15: buy2 >= 0
  R16: Convert2 >= 0
General Integers
  ['buyRa', 'buyRb', 'sellH', 'sellM', 'buy1', 'buy2', 'Convert2']
None
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 4800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 

In [17]:
if a.status == GRB.OPTIMAL:
    print('\nProfit: %g' % a.objVal)
    for v in a.getVars():
        print('%s %g' % (v.varName, v.x))
else:
    print('No solution')


Profit: 74300
buyRa 70000
buyRb 30000
sellH 43000
sellM -0
buy1 -0
buy2 0
Convert2 50000
